# 4. Unicode Text Versus Bytes
> 인간은 텍스트를 사용하고 컴퓨터는 바이트를 사용한다!!!


해당 장에서는 유니코드 문자열, 이진 시퀀스, 둘 간의 변환에 사용되는 인코딩에 대해 설명한다.

## 4. 1 문자 문제

In [1]:
s = 'café'
len(s)

4

In [5]:
b = s.encode('utf8')
print(b)
print(len(b))       # é is encoded as two bytes
print(b.decode('utf8'))

b'caf\xc3\xa9'
5
café


## 4. 2 바이트에 대한 기본 지식

In [13]:
cafe = bytes('café', encoding='utf_8')
print(cafe)

print(cafe[0])            # 각 항목은 range(256)에 들어가는 정수를 반환
print(cafe[4])
print(cafe[:1])           # bytes는 슬라이싱하면 bytes를 반환

b'caf\xc3\xa9'
99
169
b'c'


NOTE: my_bytes[0] int를 반환하지만, my_bytes[:1]은 bytes를 반환한다. 이것은 놀랍지 않다! 슬라이스는 항상 원본 시퀀스와 같은 타입을 반환하기 때문이다. s[0] == s[:1]이 되는 경우는 str타입이 유일하다.

In [14]:
cafe_arr = bytearray(cafe)
print(cafe_arr)

print(cafe_arr[-1:])    # bytearray는 슬라이싱하면 bytearray를 반환

bytearray(b'caf\xc3\xa9')
bytearray(b'\xa9')


In [20]:
print(bytes.fromhex('31 4B CE A9'))    # 16진수 문자열을 바이트 시퀀스로 변환
print(b'1K\xce\xa9'.decode())

b'1K\xce\xa9'
1KΩ


In [26]:
import array

numbers = array.array('h', [-2, -1, 0, 1, 2])    # signed short
octets = bytes(numbers)
octets

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00'

### 4. 2. 1 구조체와 메모리 뷰

In [27]:
import struct

fmt = '<3s3sHH'    # < : little-endian, 3s : 3 bytes, 3s : 3 bytes, HH : 2 shorts
with open('filter.gif', 'rb') as fp:             # filter.gif 어디있냐고...
    img = memoryview(fp.read())

header = img[:10]
bytes(header)

FileNotFoundError: [Errno 2] No such file or directory: 'filter.gif'

## 4. 3 기본 인코더/디코더

In [31]:
for codec in ['latin_1', 'cp1252', 'cp437', 'utf_8', 'utf_16']:
    print(codec, 'El Niño'.encode(codec), sep='\t')

latin_1	b'El Ni\xf1o'
cp1252	b'El Ni\xf1o'
cp437	b'El Ni\xa4o'
utf_8	b'El Ni\xc3\xb1o'
utf_16	b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


## 4. 4 인코딩/디코딩 문제 이해하기

In [33]:
city = 'São Paulo'

print(city.encode('utf_8'))
print(city.encode('utf_16'))
print(city.encode('iso8859_1'))
print(city.encode('cp437'))         # cp437에는 'ã'가 없으므로 에러 발생

b'S\xc3\xa3o Paulo'
b'\xff\xfeS\x00\xe3\x00o\x00 \x00P\x00a\x00u\x00l\x00o\x00'
b'S\xe3o Paulo'


UnicodeEncodeError: 'charmap' codec can't encode character '\xe3' in position 1: character maps to <undefined>

In [34]:
# error를 무시해보자
print(city.encode('cp437', errors='ignore'))                  # 에러 무시
print(city.encode('cp437', errors='replace'))                 # 에러가 발생하면 '?'로 대체
print(city.encode('cp437', errors='xmlcharrefreplace'))       # 에러가 발생하면 XML 문자 참조로 대체

b'So Paulo'
b'S?o Paulo'
b'S&#227;o Paulo'


### 4. 4. 2 UnicodeDecodeError 처리하기